In [ ]:
class MyKNNReg():
  def __init__(self, k = 3, metric = 'euclidean', weight = 'uniform'):
    self.k = k
    self.train_size = None
    self.metric = metric
    self.weight = weight

  def __str__(self):
    base_str = f'MyKNNReg class: k = {self.k}'
    return base_str

  def fit(self, X, y):
    self.X_train = X.copy()
    self.y_train = y.copy()
    self.train_size = X.shape

  def euclidean_distance(self, x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

  def chebyshev_distance(self, x1, x2):
    return np.max(np.abs(x1 - x2))

  def manhattan_distance(self, x1, x2):
    return np.sum(np.abs(x1 - x2))

  def cosine_distance(self, x1, x2):
    return 1 - (np.sum(x1 * x2)) / (np.sqrt(np.sum(x1 ** 2)) * np.sqrt(np.sum(x2 ** 2)))



  def _predict(self, X_test_row):
    if self.metric == 'euclidean':
      distances = self.X_train.apply(lambda X_train_row : self.euclidean_distance(X_train_row, X_test_row), axis = 1)
    elif self.metric == 'chebyshev':
      distances = self.X_train.apply(lambda X_train_row : self.chebyshev_distance(X_train_row, X_test_row), axis = 1)
    elif self.metric == 'manhattan':
      distances = self.X_train.apply(lambda X_train_row : self.manhattan_distance(X_train_row, X_test_row), axis = 1)
    elif self.metric == 'cosine':
      distances = self.X_train.apply(lambda X_train_row : self.cosine_distance(X_train_row, X_test_row), axis = 1)

    if self.weight == 'uniform':
      nearest_indexes = distances.argsort()[:self.k]
      nearest_values = self.y_train.iloc[nearest_indexes]
      return nearest_values.mean()

    elif self.weight == 'rank':
      nearest_indexes = distances.argsort()[:self.k]
      nearest_values = self.y_train.iloc[nearest_indexes]
      nearest_values = nearest_values.reset_index(drop = True)
      denom = 0
      for index, value in nearest_values.items():
        denom += 1 / (index + 1)
      weights = np.array([1 / (index + 1) / denom for index in list(nearest_values.index)])
      return np.sum(weights * nearest_values)

    elif self.weight == 'distance':
      nearest_indexes = distances.argsort()[:self.k]
      nearest_values = self.y_train.iloc[nearest_indexes]
      distances = distances.sort_values()[:self.k]
      denom = np.sum([1 / distance for distance in distances])
      weights = np.array([(1 / distance) / denom for distance in distances])
      return np.sum(weights * nearest_values)


  def predict(self, X_test):
    return X_test.apply(lambda X_test_row : self._predict(X_test_row), axis = 1)

